In [1]:
using CSV
using DataFrames
using JSON

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20210117.csv"));

In [4]:
bad_ids = begin
    id_counts = combine(groupby(rawdata, :hospital_pk), :hospital_name => (x -> length(unique(x))) => :n_names)
    filter!(x -> x.n_names > 1, id_counts)
    unique(id_counts.hospital_pk)
end
filter!(row -> !(row.hospital_pk in bad_ids), rawdata);

In [5]:
unique!(rawdata, :hospital_pk);

In [6]:
data = select(rawdata,
    :hospital_name => :hospital,
    :hospital_pk => :hospital_id,
    :state,
    :city => ByRow(x -> ismissing(x) ? x : titlecase(x)) => :city,
    :zip,
    :fips_code,
)
sort!(data, [:hospital, :hospital_id]);

In [7]:
states_data = DataFrame(CSV.File("../rawdata/states.csv"));
state_data_dict = Dict(row.abbrev => row.state for row in eachrow(states_data))
state_data_dict["GU"] = "Guam";
state_data_dict["MP"] = "Northern Mariana Islands";
state_data_dict["VI"] = "US Virgin Islands";

In [8]:
insertcols!(data, 4, :state_abbrev => deepcopy(data.state));
data.state = [haskey(state_data_dict, s) ? state_data_dict[s] : s for s in data.state];

In [9]:
hospitalsystem_rawdata = DataFrame(CSV.File("../rawdata/chsp-hospital-linkage-2018.csv"))
hospitalsystem_dict = Dict(row.ccn => (
    hospital_name = row.hospital_name,
    health_system_id = row.health_sys_id,
    health_system_name = row.health_sys_name,
) for row in eachrow(hospitalsystem_rawdata));

In [10]:
data.system_id = [haskey(hospitalsystem_dict, h) ? hospitalsystem_dict[h].health_system_id : missing for h in data.hospital_id];
data.system_name = [haskey(hospitalsystem_dict, h) ? hospitalsystem_dict[h].health_system_name : missing for h in data.hospital_id];

In [11]:
nyt_metadata = JSON.parsefile("../rawdata/nyt_hospital_metadata.json");
nyt_metadata_dict = Dict(h["ccn"] => h for h in nyt_metadata);

In [12]:
nyt_names = [haskey(nyt_metadata_dict, row.hospital_id) ? nyt_metadata_dict[row.hospital_id]["nyt_hospital_name"] : titlecase(row.hospital) for row in eachrow(data)];
insertcols!(data, 2, :hospitalname =>  nyt_names);

In [13]:
hsahrr_data = DataFrame(CSV.File("../rawdata/ZipHsaHrr18.csv"))
hsahrr_dict = Dict(row.zipcode18 => row for row in eachrow(hsahrr_data));

missing_row = (hsanum=missing, hsacity=missing, hsastate=missing, hrrnum=missing, hrrcity=missing, hrrstate=missing)
hsahrrs = [haskey(hsahrr_dict,z) ? hsahrr_dict[z] : missing_row for z in data.zip];

data.hsa_id = [h.hsanum for h in hsahrrs];
data.hsa_name = [h.hsacity * ", " * h.hsastate for h in hsahrrs];
data.hrr_id = [h.hrrnum for h in hsahrrs];
data.hrr_name = [h.hrrcity * ", " * h.hrrstate for h in hsahrrs];

In [14]:
data

,hospital,hospitalname,hospital_id,state,state_abbrev,city,zip,fips_code,system_id,system_name,hsa_id,hsa_name,hrr_id,hrr_name
,String,String,String,String,String,String?,Int64?,Int64?,String?,String?,Int64?,String?,Int64?,String?
1,ABBEVILLE AREA MEDICAL CENTER,Abbeville Area Medical Center,421301,South Carolina,SC,Abbeville,29620,45001,missing,missing,42001,"Abbeville, SC",366,"Columbia, SC"
2,ABBEVILLE GENERAL HOSPITAL,Abbeville General Hospital,190034,Louisiana,LA,Abbeville,70510,22113,HSI00000562,Lafayette General Health,19001,"Abbeville, LA",213,"Lafayette, LA"
3,ABBOTT NORTHWESTERN HOSPITAL,Abbott Northwestern Hospital,240057,Minnesota,MN,Minneapolis,55407,27053,HSI00000029,Allina Health System,24076,"Minneapolis, MN",251,"Minneapolis, MN"
4,ABILENE REGIONAL MEDICAL CENTER,Abilene Regional Medical Center,450558,Texas,TX,Abilene,79606,48441,HSI00000249,Community Health Systems,45001,"Abilene, TX",382,"Abilene, TX"
5,ABINGTON MEMORIAL HOSPITAL,Abington Memorial Hospital,390231,Pennsylvania,PA,Abington,19001,42091,HSI00000048,Jefferson Health,39001,"Abington, PA",356,"Philadelphia, PA"
6,ABRAHAM LINCOLN MEMORIAL HOSPITAL,Abraham Lincoln Memorial Hospital,141322,Illinois,IL,Lincoln,62656,17107,HSI00000648,Memorial Health System,14075,"Lincoln, IL",172,"Springfield, IL"
7,ABRAZO ARROWHEAD CAMPUS,Abrazo Arrowhead Campus,030094,Arizona,AZ,Glendale,85308,4013,HSI00001066,Tenet Healthcare Corporation,3022,"Phoenix, AZ",12,"Phoenix, AZ"
8,ABRAZO CENTRAL CAMPUS,Abrazo Central Campus,030030,Arizona,AZ,Phoenix,85015,4013,HSI00001066,Tenet Healthcare Corporation,3022,"Phoenix, AZ",12,"Phoenix, AZ"
9,ABRAZO SCOTTSDALE CAMPUS,Abrazo Scottsdale Campus,030083,Arizona,AZ,Phoenix,85032,4013,HSI00001066,Tenet Healthcare Corporation,3022,"Phoenix, AZ",12,"Phoenix, AZ"


In [15]:
data |> CSV.write("../data/hhs_hospital_meta.csv")

"../data/hhs_hospital_meta.csv"